### Перед вами датасет, описывающий поступление детей в колледж.


## целевая переменная in_college
Осмотрите данные.
Примените для предобработки пайплайн с прошлого урока. 

Решите задачу классификации методами:

1- логистической регрессии

2- линейной классификации

3- деревом решений

4- методом опорных векторов

 сравните полученные в каждом решении метрики между собой


Попробуйте использовать GridSearch для поиска оптимальных гиперпараметров, перечисленных выше алгоритмов.


In [92]:

import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, \
              auc, mean_squared_error, accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline



In [144]:
data = pd.read_csv('Churn_Modelling1.csv')
for col in data.columns:
    uniq_val = data[col].unique()
    print(f'{col} = {uniq_val}')    

data['in_college'].value_counts()



type_school = ['Academic' 'Vocational']
school_accreditation = ['A' 'B']
gender = ['Male' 'Female']
interest = ['Less Interested' 'Very Interested' 'Uncertain' 'Not Interested'
 'Quiet Interested']
residence = ['Urban' 'Rural']
parent_age = [56 57 50 49 48 52 53 47 51 60 54 55 46 58 41 59 43 42 44 64 61 40 45 62
 65]
parent_salary = [ 6950000  4410000  6500000  6600000  5250000  3770000  6680000  5890000
  6730000  3880000  5680000  5950000  6930000  7130000  3370000  6940000
  3970000  6790000  7180000  4090000  2600000  3900000  4250000  6400000
  5760000  4950000  4720000  4890000  6450000  6470000  5080000  3230000
  6130000  5700000  5900000  6610000  5940000  7310000  4660000  5630000
  5550000  5570000  2660000  4740000  7260000  7460000  6110000  5930000
  3850000  6540000  4880000  4590000  6390000  7610000  3760000  6220000
  6210000  4800000  5840000  5610000  5620000  6300000  5720000  6200000
  6700000  4100000  3200000  7570000  4200000  4500000  5750000  8390000
  655000

True     500
False    500
Name: in_college, dtype: int64

In [94]:
data.replace( {'type_school':
                 {
                  'Academic':1,
                  'Vocational':0
                 },
            'school_accreditation':
                 {
                  'A':1,
                  'B':0
                 },
            'gender': 
                 {
                   'Male': 1,
                   'Female':0    
                 },
                 
           'residence':
                {
                  'Urban':1,
                  'Rural':0
                },
           'parent_was_in_college':
                {
                   False: 0,
                   True: 1  
                },
          
          'in_college':
               {
                 True: 1,
                 False: 0
               }
           },  inplace = True)
data

,type_school,school_accreditation,gender,interest,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college,in_college
0,1,1,1,Less Interested,1,56,6950000,83.0,84.09,0,1
1,1,1,1,Less Interested,1,57,4410000,76.8,86.91,0,1
2,1,0,0,Very Interested,1,50,6500000,80.6,87.43,0,1
3,0,0,1,Very Interested,0,49,6600000,78.2,82.12,1,1
4,1,1,0,Very Interested,1,57,5250000,75.1,86.79,0,0
...,...,...,...,...,...,...,...,...,...,...,...
995,0,1,0,Very Interested,0,49,7420000,63.6,85.99,1,1
996,1,0,0,Less Interested,0,51,7480000,84.3,89.72,1,1
997,0,1,1,Less Interested,1,49,5550000,75.2,79.56,0,1
998,1,0,1,Uncertain,0,53,5840000,105.8,87.18,1,1


In [103]:
ct = ColumnTransformer(
    [("one_hot_encoder", OneHotEncoder(handle_unknown = 'ignore'), ['interest'])],
    remainder="passthrough"
)


X_transformed = ct.fit_transform(data)
names = ['Less_Interested', 'Not_Interested','Quiet_Interested', 'Uncertain' ,'Very_Interested','type_school','school_accreditation','gender','residence','parent_age','parent_salary','house_area','average_grades','parent_was_in_college','in_college']
X_transformed = pd.DataFrame(X_transformed, columns=names)


,Less_Interested,Not_Interested,Quiet_Interested,Uncertain,Very_Interested,type_school,school_accreditation,gender,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college,in_college
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,56.0,6950000.0,83.0,84.09,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,57.0,4410000.0,76.8,86.91,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,50.0,6500000.0,80.6,87.43,0.0,1.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,49.0,6600000.0,78.2,82.12,1.0,1.0
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,57.0,5250000.0,75.1,86.79,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,49.0,7420000.0,63.6,85.99,1.0,1.0
996,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,51.0,7480000.0,84.3,89.72,1.0,1.0
997,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,49.0,5550000.0,75.2,79.56,0.0,1.0
998,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,53.0,5840000.0,105.8,87.18,1.0,1.0


###  деревo

In [104]:
y = X_transformed['in_college']
x = X_transformed.drop(columns = ['in_college'])
X_train = x
Y_train = y
x_train = x.iloc[:800]
y_train = y.iloc[:800]
x_test = x.iloc[800:]
y_test = y.iloc[800:]


In [115]:
parametrs = {  'max_depth': range (1,13, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }

clf = DecisionTreeClassifier()
grid = GridSearchCV(clf, parametrs, cv=5)
grid.fit(X_train, Y_train)
grid.best_params_

{'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2}

### получается я просто подобрал параметры


In [116]:
clf_train = DecisionTreeClassifier(max_depth = 5,
 min_samples_leaf = 3,
 min_samples_split = 2)

In [117]:
clf_test = clf_train.fit(x_train,y_train)
pred_new1 = clf_test.predict(x_test)
cm_new1 = confusion_matrix(y_test, pred_new1)
print(classification_report(y_test, pred_new1), cm_new1)

              precision    recall  f1-score   support

         0.0       0.89      0.91      0.90        95
         1.0       0.91      0.90      0.90       105

    accuracy                           0.90       200
   macro avg       0.90      0.90      0.90       200
weighted avg       0.90      0.90      0.90       200
 [[86  9]
 [11 94]]


### svc

In [122]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
pipe = Pipeline([('scaler', MinMaxScaler()), ('svc', SVC())])
pipe.fit(x_train, y_train)
pred_new2 = pipe.predict(x_test)
cm_new2 = confusion_matrix(y_test, pred_new2)
print(classification_report(y_test, pred_new2), cm_new2)

              precision    recall  f1-score   support

         0.0       0.90      0.91      0.90        95
         1.0       0.91      0.90      0.91       105

    accuracy                           0.91       200
   macro avg       0.90      0.91      0.90       200
weighted avg       0.91      0.91      0.91       200
 [[86  9]
 [10 95]]


### линейной классификации

In [132]:
from sklearn.linear_model import LinearRegression



In [133]:
pipe2 = Pipeline([('scaler', MinMaxScaler()), ('lin', LinearRegression())])

In [135]:
pipe2.fit(x_train,y_train)
pred_new3 = pipe.predict(x_test)
cm_new3 = confusion_matrix(y_test, pred_new3)
print(classification_report(y_test, pred_new3), cm_new3)

              precision    recall  f1-score   support

         0.0       0.90      0.91      0.90        95
         1.0       0.91      0.90      0.91       105

    accuracy                           0.91       200
   macro avg       0.90      0.91      0.90       200
weighted avg       0.91      0.91      0.91       200
 [[86  9]
 [10 95]]


### логистическая регрессия

In [138]:
from sklearn.linear_model import LogisticRegression
pipe3 = Pipeline([('scaler', MinMaxScaler()), ('lin', LogisticRegression())])
pipe3.fit(x_train,y_train)
pred_new4 = pipe.predict(x_test)
cm_new4 = confusion_matrix(y_test, pred_new4)
print(classification_report(y_test, pred_new4), cm_new4)



              precision    recall  f1-score   support

         0.0       0.90      0.91      0.90        95
         1.0       0.91      0.90      0.91       105

    accuracy                           0.91       200
   macro avg       0.90      0.91      0.90       200
weighted avg       0.91      0.91      0.91       200
 [[86  9]
 [10 95]]


### результаты одинаковые